In [46]:
# Script:
# 1. Make a new GoogleSheet file
# 2. Save a few values from previous Timesheet:
#   - B (column) X (row): Accumulated yearly salary based on hrs only
# 3. Generate a list of working days strictly within that month
#    - goes into A1:An; where n is len(working_days)
# 4. Fill in B1:Bn with default number: 8
#    - add the already spread (equally, as possible) excess/missing working hrs
# 5. Rename so it has the one it should (increment the invoice_id compared to the previous one)

In [101]:
import re
import sys
import traceback
from datetime import date, timedelta, datetime
from enum import Enum
from typing import Optional, List, Union

import gspread
from dateutil.relativedelta import relativedelta

In [104]:
# %config IPCompleter.use_jedi=False # Necessary only if it is being developed on self-hosted page
class SpreadMode(Enum):
    even = "spread evenly"
    consider_week_numbers = "squeeze within specific weeks"

In [105]:
#                                                         #
# Initial setup for every run, that needs to be included: #
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # #

# if you leave `None`: generates for the last month (<=> current_month - 1)
PREVIOUS_MONTH_AS_NUMBER: Optional[int] = 6  # if not None then should be between: [1..12]

# Declare other items that need to be invoiced, such as food, electricity, bonus, etc...
other_items = [
    ("Other(food)", 100_000, "EUR"),
    ("Works above agreed volume", 1_000_000, "EUR"),
]  # (Name, value, unit)

# Work in excess or in minus - (sur-)plus_hrs
#  could be a sole number too, but if not, use TAB as separator (or change the separator below)
# & if some of those are negative, can be interpreted as skipped hrs/workdays even
excess_work_input: str = "0	8	0	17.5	8.5"  # for e.g. week 1st (only 1st of July) has 0 surplus, week 2nd (days of July 04.->08.) has 8hrs (1day), ..., week 5th (days of July 25.->29.) has 8.5hrs
excess_work_input_separator: str = "\t"
# Choose in which mode you want to spread the excess/lost amount of hours across the working days
SPREAD_MODE = SpreadMode.even

In [50]:
#                                                                 #
# Initial setup that doesn't change too much during series of run #
# # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
FULL_UNDERSCORED_NAME = "JONES_DOW"

INVOICE_ID_PREFIX = "DOW"  # it's usually a constant, e.g.: DOW0001

SALARY_PER_HOUR = 150  # don't let anybody know
#
######
#
NUMBER_OF_WORKING_DAYS_PER_WEEK = 5
MAX_NUMBER_OF_EXCESS_WORKING_HOURS_PER_DAY = 2

In [108]:
def spread_excess_work_hrs(working_hrs_per_day: List[float], working_hrs_in_excess_or_loss: Union[int, List],
                           spread_mode: SpreadMode, no_workingdays_per_week=5,
                           max_no_workinghrs_per_day=2) -> List[float]:
    if spread_mode == SpreadMode.even:
        excess_to_be_rolled = sum(working_hrs_in_excess_or_loss)
    elif spread_mode == SpreadMode.consider_week_numbers:
        raise NotImplementedError(f"No spread mode implementation for: {spread_mode}")
    else:
        raise NotImplementedError(f"No spread mode implementation for: {spread_mode}")

    for i, working_hrs in enumerate(working_hrs_per_day):
        if 0.1 > excess_to_be_rolled >= 0:
            break
        if working_hrs < no_workingdays_per_week * max_no_workinghrs_per_day:
            val = no_workingdays_per_week * max_no_workinghrs_per_day - working_hrs
            working_hrs_per_day[i] += val
            excess_to_be_rolled -= val if excess_to_be_rolled > 0 else -val

    return working_hrs_per_day

In [52]:
# Declare filename templates
PREVIOUS_INVOICE_ID = PREVIOUS_MONTH_AS_NUMBER if PREVIOUS_MONTH_AS_NUMBER else (
        datetime.now() - relativedelta(months=1)).month
print("Previous invoice ID: '{}'".format(PREVIOUS_INVOICE_ID))

CURRENT_YEAR = datetime.now().year
GET_INVOICE_FILENAME_TEMPLATE = lambda invoice_id, year, month: "{}_invoice_{}{}_{}-{}".format(FULL_UNDERSCORED_NAME,
                                                                                               INVOICE_ID_PREFIX,
                                                                                               invoice_id, year, month)

MONTHLY_ACCUMULATED_HRS_CELL_VALUE = "* Surplus (accumulated) hrs"
TOT_CELL_VALUE = "TOT."
YEARLY_PAID_WORKING_HRS_ONLY_CELL_VALUE = "* Total/year(only paid working hrs.):"

GET_SUM_ACCUMULATED_BY_WORKINGHRS_ONLY = lambda sheet: sheet.cell(
    sheet.find(YEARLY_PAID_WORKING_HRS_ONLY_CELL_VALUE).row, 2).value


Previous invoice ID: '6'


In [53]:
gc = gspread.service_account()

# Let's open the necessary (previous invoice) file
all_g_spreadsheets = map(lambda x: x.title, gc.openall())

prev_invoice_filename = next(
    filter(lambda x: re.match(GET_INVOICE_FILENAME_TEMPLATE(*[".*"] * 2, PREVIOUS_INVOICE_ID), x), all_g_spreadsheets))
prev_file = gc.open(prev_invoice_filename)
prev_sheet = gc.open(prev_invoice_filename).sheet1


def delete_g_spreadsheets():
    """Call the method, if you want to delete files on the Drive"""
    all_g_spreadsheets = map(lambda x: x, gc.openall())
    spreadsheet_files_to_delete = (
        filter(lambda x: re.match(GET_INVOICE_FILENAME_TEMPLATE(*[".*"] * 2, 7), x.title), all_g_spreadsheets))
    [gc.del_spreadsheet(x.id) for x in spreadsheet_files_to_delete]


In [54]:
from typing import List, Union, Optional
from dataclasses import dataclass


@dataclass
class WorkdayWorkedHrs:
    dated_day: datetime
    hrs_worked: int
    row: int = 2

    def to_excel_format(self) -> List[Union[str, int]]:
        return [str(self.dated_day), self.hrs_worked]


# NOTE: Do not configure padding as its dynamic padding is broken
class Paddable:
    """Could possible have: `pad`, `pad_before`, `pad_after`"""

    @staticmethod
    def pad_values_if_possible(paddable_instance):
        # TODO: Fix padding to be dynamic
        result = []
        if getattr(paddable_instance, "pad_before", None):
            result += [*[[]] * paddable_instance.pad_before,
                       [paddable_instance.name, paddable_instance.form, paddable_instance.unit]]
        elif getattr(paddable_instance, "pad", None):
            result += [*[[]] * paddable_instance.pad,
                       [paddable_instance.name, paddable_instance.form, paddable_instance.unit],
                       *[[]] * paddable_instance.pad]
        elif getattr(paddable_instance, "pad_after", None):
            result += [[paddable_instance.name, paddable_instance.form, paddable_instance.unit],
                       *[[]] * paddable_instance.pad_after]
        return result


@dataclass
class NameFormUnit:
    name: str
    form: str
    unit: Union[str, int] = "EUR"
    row: Optional[int] = None

    def to_excel_format(self) -> List[str]:
        return [self.name, self.form, self.unit]


@dataclass
class TotalHrs(NameFormUnit, Paddable):
    name: str = TOT_CELL_VALUE
    form: str = "=SUM(B2:B{})"
    unit: str = "Hours"
    pad: int = 1


@dataclass
class TotalSalaryPerHrsOnly(NameFormUnit):
    name: str = "Amount / working hours"
    form: str = "={}*SUM(B2:B{})"


@dataclass
class Other(NameFormUnit): pass


@dataclass
class TotalToBePaid(NameFormUnit, Paddable):
    name: str = "TOT. to be paid"
    form: str = "=SUM(B{}:B{})"
    pad: int = 1


@dataclass
class Notice(NameFormUnit): pass


@dataclass
class Invoice:
    column = ["Workdays only", "Hours / day"]
    workdays_worked: List[WorkdayWorkedHrs]
    total_hrs: TotalHrs
    tot_salary_hrs_only: TotalSalaryPerHrsOnly
    others: List[Other]
    tot_tobe_paid: TotalToBePaid

    notices: List[Notice]

    def to_list(self) -> list:
        result = [self.column]
        result.extend(list(map(WorkdayWorkedHrs.to_excel_format, self.workdays_worked)))
        result.extend(self.total_hrs.pad_values_if_possible(self.total_hrs))
        result.extend([self.tot_salary_hrs_only.to_excel_format()])
        result += list(map(NameFormUnit.to_excel_format, self.others))
        result.extend(self.tot_tobe_paid.pad_values_if_possible(self.tot_tobe_paid))
        result.extend(map(NameFormUnit.to_excel_format, self.notices))
        return result

    def format_spreadsheet(self, sh: gspread.Spreadsheet.worksheet):
        # TODO: rewrite formatting so it becomes more dynamic and elegant
        sh.format("A1:D1", {
            "horizontalAlignment": "CENTER",
            "textFormat": {
                "fontSize": 12,
                "bold": True
            }
        })

        row = self.workdays_worked[0].row + 2
        sh.format("A{}:D{}".format(row, row), {
            "backgroundColor": {
                "red": 56 / 255,
                "green": 163 / 255,
                "blue": 168 / 255
            },
            "horizontalAlignment": "CENTER",
            "textFormat": {
                "fontSize": 12,
                "bold": True
            }
        })

        row = self.tot_salary_hrs_only.row + len(self.others) if self.others else self.tot_salary_hrs_only.row
        sh.format("A{}:D{}".format(self.tot_salary_hrs_only.row, row), {
            "backgroundColor": {
                "red": 156 / 255,
                "green": 214 / 255,
                "blue": 48 / 255
            }})

        row += 1
        sh.format("A{}:D{}".format(row, row), {
            "backgroundColor": {
                "red": 0 / 255,
                "green": 0 / 255,
                "blue": 0 / 255
            }})

        row += 1
        sh.format("A{}:D{}".format(row, row), {
            "backgroundColor": {
                "red": 51 / 255,
                "green": 165 / 255,
                "blue": 59 / 255
            },
            "textFormat": {
                "fontSize": 12,
                "bold": True
            }
        })


In [112]:
# Build the invoice from scratch

def get_date_for_current_invoice(invoice_filename_by_convention: str) -> date:
    """ return: first day of current year's following month to last invoice """
    return date(datetime.now().year, int(invoice_filename_by_convention[-1]), 1) + relativedelta(months=1)


# Workdays with working hrs.

# Get list of all days
invoice_date = get_date_for_current_invoice(prev_invoice_filename)
all_days = (invoice_date + timedelta(x) for x in range(((invoice_date + relativedelta(months=1)) - invoice_date).days))
workdays = list(filter(lambda d: d.weekday() + 1 <= NUMBER_OF_WORKING_DAYS_PER_WEEK, all_days))

work_in_plus_per_week: List[float] = list(map(float, excess_work_input.split("\t")))

TOT_WORK_IN_PLUS = sum(work_in_plus_per_week)
# if TOT_WORK_IN_PLUS:

# work_in_plus_per_workdays_ = [8] * len(workdays)
working_hrs_with_excess_or_loss_per_day: List[float] = [8.0] * len(workdays)
working_hrs_with_excess_or_loss_per_day = spread_excess_work_hrs(working_hrs_with_excess_or_loss_per_day,
                                                                 work_in_plus_per_week, SPREAD_MODE)
if len(workdays) * 8 + TOT_WORK_IN_PLUS != sum(
        working_hrs_with_excess_or_loss_per_day):  # it means `MAX_NUMBER_OF_EXCESS_WORKING_HOURS_PER_DAY` is not enough
    raise SystemError("Leaking working hour calculation in the system, please look out!")

ww = [WorkdayWorkedHrs(d, worked_hrs_with_excess) for d, worked_hrs_with_excess in
      zip(workdays, working_hrs_with_excess_or_loss_per_day)]
ww_row = len(ww) + 1

print(len(workdays), len(work_in_plus_per_week), working_hrs_with_excess_or_loss_per_day,
      len(workdays) * 8 + TOT_WORK_IN_PLUS, sum(working_hrs_with_excess_or_loss_per_day))

21 5 [10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 10.0, 8.0, 8.0, 8.0, 8.0] 202.0 202.0


In [222]:

# Tot.Hrs.
tot_hrs = TotalHrs()
tot_hrs.form = tot_hrs.form.format(ww_row)

# Tot.Salary/Hrs.Only
tot_salary_hrs_only = TotalSalaryPerHrsOnly()
tot_salary_hrs_only.form = tot_salary_hrs_only.form.format(SALARY_PER_HOUR, ww_row)
tot_salary_hrs_only.row = ww_row + 4

# Others [if are]

others = list(map(lambda x: Other(*x), other_items))

# Tot.ToBePaid
tot_tobe_paid = TotalToBePaid()
tot_tobe_paid.row = ww_row + 4 + len(others)
tot_tobe_paid.form = tot_tobe_paid.form.format(tot_salary_hrs_only.row, tot_tobe_paid.row)

# All notices here
worked_hrs_in_excess = sum(map(lambda x: x.hrs_worked, ww)) - len(workdays) * 8
additional_notices = [
    (YEARLY_PAID_WORKING_HRS_ONLY_CELL_VALUE,
     f"=B{tot_salary_hrs_only.row} + {GET_SUM_ACCUMULATED_BY_WORKINGHRS_ONLY(prev_sheet)}",
     "EUR"),
    (MONTHLY_ACCUMULATED_HRS_CELL_VALUE,
     # You could use GSheet formula too
     # f"""=IF(Sum(B2:B{ww_row}) > 8*NETWORKDAYS(A2,EOMONTH(A2,0),E3:E15), Sum(B2:B{ww_row})-8*NETWORKDAYS(A2,EOMONTH(A2,0),E3:E15), "No accumulated hrs in plus this month")""",
     worked_hrs_in_excess if worked_hrs_in_excess else "No accumulated hrs in plus this month",
     "")]

notices = list(map(lambda x: Notice(*x), additional_notices))

# Finally, our invoice
invoice = Invoice(ww, tot_hrs, tot_salary_hrs_only, others, tot_tobe_paid, notices)


list of: 202.0, total hrs: 168; TOT: 34.0


In [223]:
# Say no more, ready to create the new invoice calulcator
# Extract
def extract_invoice_id_from(invoice_filename: str, invoice_id_prefix) -> str:
    return re.findall(rf"(?<=_{invoice_id_prefix})\d+(?=_)", invoice_filename)[0]


def increment_invoice_id(invoice_id: str) -> str:
    return str(int(invoice_id) + 1).zfill(len(invoice_id))


invoice_filename = GET_INVOICE_FILENAME_TEMPLATE(
    increment_invoice_id(extract_invoice_id_from(prev_invoice_filename, INVOICE_ID_PREFIX)),
    invoice_date.year, invoice_date.month)

sh = gc.create(invoice_filename, "1xyTWWbq6CiwTmNChx6UF3gMQax4talU2")

print("Previous file's permissions: ", gc.list_permissions(prev_file.id))
gc.insert_permission(
    sh.id,
    'portikHun@gmail.com', perm_type='user', role='writer'
)

list_values = invoice.to_list()
try:
    invoice.format_spreadsheet(sh.sheet1)
    sh.sheet1.update('A1:E{}'.format(len(list_values)), list_values, raw=False)
    invoice.format_spreadsheet(sh.sheet1)
except Exception as e:
    print(traceback.format_exc(), file=sys.stderr)
    print("Error found, deleting the newly created GSheet!")
    gc.del_spreadsheet(sh.id)

print("Everything is DONE!")

first row: 24
first row: 24
Everything is DONE!


In [1]:
# TODO: supervise if it's necessary to be used
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)